In [8]:
import numpy as np
import cv2
import matplotlib.pylab as plt
import math
import statistics

In [9]:
img = cv2.imread("book.jpg")
#image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
i=img.shape[0]
j=img.shape[1]
image = np.zeros(shape=(img.shape[0],img.shape[1]))
for x in range(i):
    for y in range(j):
        image[x,y]=img[x,y].mean()
print(image.shape)

(264, 220)


In [10]:
result = np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
r= result.shape[0]
c= result.shape[1]
fx = np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
fy = np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
GM = np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
GD = np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
NMS = np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
DT =  np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
Hys =np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
print(GM.shape)
print(GD.shape)
print(NMS.shape)

(266, 222)
(266, 222)
(266, 222)


In [11]:
SobelX=(np.array([[-1,0,1],[-2,0,2],[-1,0,1]]))
SobelY=(np.array([[-1,-2,-1],[0,0,0],[1,2,1]]))
Gaussian=(1/16)*(np.array([[1,2,1],[2,4,2],[1,2,1]]))

In [12]:
#Generic convolution code
def Convolution(img,kernel):


    #storing size of kerel
    rK=kernel.shape[0]
    cK=kernel.shape[1]
    #Horizaontal and vertical flipping of kernel
    np.flip(kernel,axis=0)
    np.flip(kernel,axis=1)
    #find the num of rows and colums to be padded
    newSize=math.floor(rK/2)
    #applying padding
    imge=np.pad(img, newSize, mode='constant')
    #temporary array
    resultant=np.zeros(shape=(imge.shape[0],imge.shape[1]))
    #storing sizes of padded image
    ri=imge.shape[0]
    ci=imge.shape[1]
    #looping for convolution
    for i in range(ri):
      for j in range(ci):
        sum =0
        if((i>(newSize-1) and i<ri-(newSize) ) and (j>(newSize-1) and j<ci-(newSize) )):
          x=i-newSize
          for l in range(rK):
            y=j-newSize
            for m in range(cK):
              sum += imge[x,y]*kernel[l,m]
              y+=1
            x+=1
          resultant[i,j]=sum
    np.clip(resultant,0,255)
    return resultant

In [16]:
def noise_reduction():
    result = Convolution(image,Gaussian)
    cv2.imwrite("GaussianSmoothening.jpg",result)
    return result

def derivative_calculation(result):
    fx= Convolution(result,SobelX)
    cv2.imwrite("fx.jpg",fx)
    fy = Convolution(result,SobelY)
    cv2.imwrite("fy.jpg",fy)
    return fx,fy

def gradient_calculation(fX,fY):
    gm = np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
    gd = np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
    val=0
    #looping for Gradient Mangnitude
    for i in range(r):
      for j in range(c):
          #magnitude image stored in resultant image
          FX=fX[i,j]*fX[i,j]
          FY=fY[i,j]*fY[i,j]
          sum=FX+FY
          val =sum**(1/2)
          gm[i,j]= val
    np.clip(gm,0,255)
    gd= np.arctan2( fY , fX )
    gd= np.rad2deg(gd)
    gd += 180;
    return gm,gd


def non_maximum_suppression(GD,GM):
    r=GM.shape[0]
    c=GM.shape[1]
    nms = np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
    for i in range (r-1):
        for j in range(c-1):
            if ((GD[i,j] > 22.5 and GD[i,j] <= 67.5) or (GD[i,j] > 202.5 and GD[i,j] <= 247.5)):
                if (GM[i,j] < GM[i-1,j+1] or GM[i,j] < GM[i+1,j-1]):
                    nms[i,j]=0
                else: 
                    nms[i,j]=GM[i,j]
            elif ((GD[i,j] > 67.5 and GD[i,j] <= 112.5) or (GD[i,j] > 247.5 and GD[i,j] <= 292.5)):
                if (GM[i,j] < GM[i-1,j] or   GM[i,j] < GM[i+1,j]):
                    nms[i,j]=0
                else: 
                    nms[i,j]=GM[i,j]
            elif ((GD[i,j] > 112.5 and GD[i,j] <= 157.5) or (GD[i,j] > 292.5 and GD[i,j] <= 337.5)):
                if (GM[i,j] < GM[i-1,j-1] or   GM[i,j] < GM[i+1,j+1]):
                    nms[i,j]=0
                else: 
                    nms[i,j]=GM[i,j]
            elif ((GD[i,j] > 157.5 and GD[i,j] <= 202.5) or (GD[i,j] > 337.5 and GD[i,j] <= 22.5)):
                if (GM[i,j] < GM[i,j-1] or   GM[i,j] < GM[i,j+1]):
                    nms[i,j]=0
                else: 
                    nms[i,j]=GM[i,j] 
    return nms               



def double_threshold(nms):
    print("1")
    dt =  np.zeros(shape=(image.shape[0]+2,image.shape[1]+2))
    print("2")
    mean = nms.mean();
    print("3")
    variance = statistics.variance(nms)
    print("4")
    maximum = nms.max()
    print("5")
    a=6
    b=3
    #Read a journal to find this formula to get best tlow and thigh value its link is below
    #https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.740.7907&rep=rep1&type=pdf
    print("6")
    val=(int)((mean-variance)/a)
    print("7")
    val2=(int)((mean+variance)/b)
    print("8")
    tlow =max(0,val)
    print("9")
    thigh =min(maximum,val2)
    for i in range(r):
        for j in range(c):
            if(nms[i,j] > thigh):
                dt[i,j] = 255
            elif(nms[i,j] < tlow):
                dt[i,j] = 0
            elif(nms[i,j] <= thigh and nms[i,j] >= tlow):
                dt[i,j] = 50
    return dt            

def hysteresis(dt,hys):
    for i in range(r):
        for j in range(c):
            if(dt[i,j]==50):
                if(dt[i-1,j-1]==255 or  dt[i-1,j] ==255 or dt[i-1,j+1]==255 or  dt[i,j-1 ] ==255 or dt[i,j+1]==255 or  dt[i+1,j-1] ==255 or dt[i+1,j] ==255 or dt[i+1,j+1]==255):
                    hys[i,j]==255
                else:
                    hys[i,j]=dt[i,j]
            else:
                hys[i,j]=dt[i,j]
    cv2.imwrite("hysterisisThreshold.jpg",hys)
    return hys
    
def main():
    result=noise_reduction()
    fx,fy=derivative_calculation(result)
    GM,GD= gradient_calculation(fx,fy)
    cv2.imwrite("GradientMagnitude.jpg",GM)
    cv2.imwrite("GradientDirection.jpg",GD)
    NMS=non_maximum_suppression(GD,GM)
    cv2.imwrite("NMS.jpg",NMS)
    DT=double_threshold(NMS)
    cv2.imwrite("DoubleThresholding.jpg",DT)
    #hysteresis(DT,Hys)

main()

TypeError: can't convert type 'ndarray' to numerator/denominator